In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

In [2]:
city_dict = pd.read_excel('Data.xlsx', sheet_name = 'city_dict')
data = pd.read_excel('Data.xlsx', sheet_name = 'Dataset')

In [3]:
df = data.merge(city_dict, on = 'id_city', how = 'left')

In [4]:
# Проверяем наличие пропусков.
df.info()
print(df.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21939 entries, 0 to 21938
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id_view      21939 non-null  int64         
 1   time_view    21939 non-null  datetime64[ns]
 2   id_group     21939 non-null  int64         
 3   id_city      21939 non-null  int64         
 4   nflag_order  21939 non-null  int64         
 5   name_city    21939 non-null  object        
dtypes: datetime64[ns](1), int64(4), object(1)
memory usage: 1.0+ MB
id_view        0
time_view      0
id_group       0
id_city        0
nflag_order    0
name_city      0
dtype: int64


In [5]:
# Проверяем наличие дубликатов. Дубликатов не было.
df.drop_duplicates()

,id_view,time_view,id_group,id_city,nflag_order,name_city
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск
1,10457152884,2023-01-30 17:38:18.629,1,1,1,Москва
2,10457162947,2023-01-07 14:06:22.689,1,1,1,Москва
3,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск
4,10457176480,2023-01-04 22:56:19.240,0,1,1,Москва
...,...,...,...,...,...,...
21934,10457187341,2023-01-07 01:09:40.477,1,1,0,Москва
21935,10457168816,2023-01-21 02:42:09.559,0,1,1,Москва
21936,10457173452,2023-01-13 12:57:11.272,1,1,1,Москва
21937,10457142488,2023-01-24 21:53:01.634,1,8,1,Нижний Новгород


In [6]:
# Проверяем данные на выбросы.
data.describe()

,id_view,time_view,id_group,id_city,nflag_order
count,2.193900e+04,21939,21939.000000,21939.000000,21939.000000
mean,1.045717e+10,2023-01-16 11:49:26.647911936,0.506541,2.762979,0.645836
min,1.045712e+10,2023-01-01 00:02:13.376000,0.000000,1.000000,0.000000
25%,1.045715e+10,2023-01-08 14:16:15.535000064,0.000000,1.000000,0.000000
50%,1.045717e+10,2023-01-16 13:22:53.428999936,1.000000,2.000000,1.000000
75%,1.045719e+10,2023-01-24 05:31:15.156000,1.000000,3.000000,1.000000
max,1.045720e+10,2023-01-31 23:57:58.622000,1.000000,11.000000,1.000000
std,2.114100e+04,NaN,0.499969,2.414341,0.478270


In [7]:
# Убедимся, что каждый пользователь встречается только в одной группе: либо в контрольной, либо в тестовой.
clients_gr = df.groupby('id_view').agg({'nflag_order': 'nunique'}).sort_values('nflag_order', ascending = False).reset_index()
clients_gr = clients_gr[clients_gr['nflag_order'] == 2]
clients_gr

,id_view,nflag_order


In [8]:
# Посмотрим метрики для анализа после очистки данных.
df_control = df[df['id_group']==0]
df_test = df[df['id_group']==1]

print('Конверсии')
print(df_control['nflag_order'].mean())
print(df_test['nflag_order'].mean())

print('\n')
print('Количество наблюдений')
print(df_control['id_view'].count())
print(df_test['id_view'].count())

Конверсии
0.6150932939220395
0.6757851165301899


Количество наблюдений
10826
11113


In [9]:
# Расчет результатов A/B-теста.

In [10]:
def test_calc (r1, r2, alpha = 0.05):
    s, p = ttest_ind(r1,r2)
    s, p
    if p <= alpha:
         if r1.mean() > r2.mean():
             print(s, "pvalue:", p)
             print('Гипотеза отвергается, среднее в тестовой группе выше')
         else: 
             print(s, "pvalue:", p)
             print('Гипотеза отвергается, среднее в контрольной группе выше')
    else: 
        print(s, "pvalue:", p)
        print('Гипотеза не отвергается, средние не значимо различаются')

In [11]:
test_calc (r1 = df_test['nflag_order'], r2 = df_control['nflag_order'], alpha = 0.05)

9.41595134321716 pvalue: 5.136595732522053e-21
Гипотеза отвергается, среднее в тестовой группе выше


In [12]:
abc = []

for i in df['name_city'].unique():
    s, p = ttest_ind(df_test[df_test['name_city'] == i]['nflag_order'], df_control[df_control['name_city'] == i]['nflag_order'])
    if p<=0.05:
        abc.append(i)
abc

['Москва', 'Санкт-Петербург', 'Казань']

In [13]:
df['hour'] = df['time_view'].dt.hour
df['flag_hour_order'] = np.where(df['hour'].isin([7,8,9,10,17,18,19,20]), 1, 0)
df

,id_view,time_view,id_group,id_city,nflag_order,name_city,hour,flag_hour_order
0,10457162393,2023-01-17 13:56:06.118,0,3,1,Новосибирск,13,0
1,10457152884,2023-01-30 17:38:18.629,1,1,1,Москва,17,1
2,10457162947,2023-01-07 14:06:22.689,1,1,1,Москва,14,0
3,10457197478,2023-01-02 11:02:58.189,0,3,0,Новосибирск,11,0
4,10457176480,2023-01-04 22:56:19.240,0,1,1,Москва,22,0
...,...,...,...,...,...,...,...,...
21934,10457187341,2023-01-07 01:09:40.477,1,1,0,Москва,1,0
21935,10457168816,2023-01-21 02:42:09.559,0,1,1,Москва,2,0
21936,10457173452,2023-01-13 12:57:11.272,1,1,1,Москва,12,0
21937,10457142488,2023-01-24 21:53:01.634,1,8,1,Нижний Новгород,21,0


In [14]:
print("Час-пик")
test_calc (r1 = df[(df['flag_hour_order'] == 1) & (df['id_group'] == 1)]['nflag_order'] , \
r2 = df[(df['flag_hour_order'] == 1) & (df['id_group'] == 0)]['nflag_order'], alpha = 0.05)
print()
print("Не час-пик")
test_calc (r1 = df[(df['flag_hour_order'] == 0) & (df['id_group'] == 1)]['nflag_order'] , \
r2 = df[(df['flag_hour_order'] == 0) & (df['id_group'] == 0)]['nflag_order'], alpha = 0.05)

Час-пик
6.422002058720224 pvalue: 1.429452278435098e-10
Гипотеза отвергается, среднее в тестовой группе выше

Не час-пик
6.997243317124079 pvalue: 2.7237344158581742e-12
Гипотеза отвергается, среднее в тестовой группе выше


In [15]:
#На основании проведенного АВ теста можно сделать вывод, что понижение цены влияет на конверсию из просмотра в создание заказа. 
#Более детально рассмотрев результаты исследования по городам, можно сделать вывод, что значимое влияние оказалось в городах:
#Москва, Санкт-Петербург, Казань. Проверка пиковых и непиковых часов показала одинаковый результат, а именно, 
#понижение цены влияет на конверсию из просмотра в создание заказа.
#По результатам теста рекомендуется введение новой платежной механики, которая должна немного понизить цену на услугу в некоторых городах.
#Для остальных сделать дополнительный тест. 